In [62]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import os
import numpy as np
from PIL import Image

In [107]:
# Define the CNN architecture
class CNN(nn.Module):
    def __init__(self, device, image_sizes= 299, kernel_size=3, max_pool_size=2, lr=0.001, epochs=3):
        """
        CNN contructor
        inputs:
        device - device in which the cnn will be executed
        kernel_size (hiperparameter) - this is the size that will be used in the convolution layers
            type: int, a single number will work with a grid of size n x n
        max_pool_size - this is the size of the pooling layer
        lr (hiperparameter) - the learning rate that will be used to train the cnn
        epochs (hiperparmeter) - the amount of iterations that will be used to train the cnn
        """
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=kernel_size, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=max_pool_size, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=kernel_size, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=max_pool_size, stride=2)
        self.fc1 = nn.Linear(32 * (image_sizes//(max_pool_size**2))**2, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 4)

        self.lr = lr
        self.epochs = epochs
        self.device = device


    def forward(self, x):
        """
        The forward function is used in the training to understand the infrastructure
        of the proposed model
        """
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        return x
    
    def train_cnn(self, train_dataset, train_labels):
        """
        Method to train the cnn based on the inputs:
        train_dataset - tensor with the information of the pixels of the images
        train_labels - tesnro with the category of each of the images frrom the train_dataset
        """
        # Set the loss function and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.parameters(), self.lr)

        # Create a data loader for the training dataset
        train_loader = DataLoader(dataset=list(zip(train_dataset, train_labels)), batch_size=16, shuffle=True)

        self.train()
        for epoch in range(self.epochs):
            running_loss = 0.0
            for images, labels in train_loader:
                images = images.to(self.device)
                labels = labels.to(self.device)

                optimizer.zero_grad()

                outputs = self(images)
                loss = criterion(outputs, labels)

                loss.backward()
                optimizer.step()

            running_loss += loss.item() * images.size(0)

            epoch_loss = running_loss / len(train_dataset)
            print(f"Epoch [{epoch+1}/{self.epochs}], Loss: {epoch_loss:.4f}")

    def predict(self, test_dataset):
        predictions = []
        self.eval()  # Set the model to evaluation mode
        # Create a data loader for the training dataset
        dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True)

        with torch.no_grad():  # Disable gradient computation for efficiency
            for inputs in dataloader:
                # Forward pass through the model to obtain predictions
                outputs = self(inputs)
                # Get the predicted class indices
                _, predicted = torch.max(outputs, 1)
                # Append the predicted class indices to the predictions list
                predictions.extend(predicted.tolist())
        
        return predictions


In [124]:
# Feature engineering - crop images borders

original_images_folder = 'images/original_dataset/'
cropped_images_folder = 'images/cropped_dataset/'

target_size = (250, 250)

categories = {'COVID': 0, 'Lung_Opacity': 1, 'Normal': 2, 'Viral_Pneumonia': 3}

for category in categories.keys():

    cat_files = os.listdir(os.path.join(original_images_folder, category))

    for file in cat_files:
        # constructing image path
        input_path = os.path.join(original_images_folder, category, file)
        # Open the image
        image = Image.open(input_path)
        # get original image size and calculate borders
        width, height = image.size

        left = (width - target_size[0]) // 2
        upper = (height - target_size[1]) // 2
        right = left + target_size[0]
        lower = upper + target_size[1]

        # crop the image
        cropped_image = image.crop((left, upper, right, lower))
        # construct the new path
        output_path = os.path.join(cropped_images_folder, category, file)
        cropped_image.save(output_path)
        image.close()

In [ ]:
# Feature engineering - getting averaged images 



In [95]:
parent_folder_path = 'images/Radiography_Dataset/'

categories = {'COVID': 0, 'Lung_Opacity': 1, 'Normal': 2, 'Viral_Pneumonia': 3}

arrays = []
image_labels = []
category_amount = []

for cat_folder, value in categories.items():

    folder_path = os.path.join(parent_folder_path, cat_folder)
    image_files = os.listdir(folder_path)
    category_amount.append(len(image_files))


    for i, file_name in enumerate(image_files):

        if i > 50: break

        image_labels.append(value)

        file_path = os.path.join(folder_path, file_name)
        # open image using PIL
        image = Image.open(file_path)
        # image to numpy array
        image_array = np.array(image)

        if image_array.shape != (299,299):
            continue

        arrays.append(image_array)

image_data = np.stack(arrays, axis=0)
image_data = image_data.reshape(len(image_data), 1, 299, 299)
image_labels = np.array(image_labels)

print(image_data.shape)
print(image_labels)
print(category_amount)

(201, 1, 299, 299)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
[3616, 6012, 10192, 1345]


In [111]:
# Create an instance of the CNN model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CNN(epochs=4, device=device, image_sizes=299).to(device)

model.train_cnn(torch.from_numpy(image_data).to(torch.float32), torch.from_numpy(image_labels).to(torch.long))


model.predict(torch.from_numpy(image_data[51:55]).to(torch.float32))

Epoch [1/4], Loss: 3.0896
Epoch [2/4], Loss: 0.3157
Epoch [3/4], Loss: 0.0112
Epoch [4/4], Loss: 0.0051


[1, 1, 1, 1]